[View in Colaboratory](https://colab.research.google.com/github/fatmas1982/fh2017/blob/master/fetch_20newsgroups2_9.ipynb)

# Load All API

In [7]:
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 
import numpy as np
import scipy.stats.stats as st
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#from stemming.porter2 import stem
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv

import re
from nltk import word_tokenize
import string
!pip install gensim
import  gensim.models as md
from gensim.models.phrases import Phrases, Phraser
import os
from sklearn.datasets import fetch_20newsgroups
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic as wnic
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Authosentecat to Google Drive

In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
#https://colab.research.google.com/notebook#fileId=1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q&scrollTo=c99EvWo1s9-x
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!ls

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!mkdir -p drive_PH
!google-drive-ocamlfuse drive_PH

In [0]:
!ls 'drive_PH/Colab Notebooks'

# Start My Code

## Settings **Codes**

In [0]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

In [0]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs

In [0]:
def read_text_from_database(path_database,file_databbase):
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph

In [0]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):#header_list=['index','text']
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line]#,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()

In [0]:
def pragraph_to_setnences(str):
    return sent_tokenize(str)

In [0]:
import csv   
def add_row(row,path_database,file_name):
    #fields=['first','second','third']
    with open(path_database+file_name, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        print("printed")

## pathes and files

In [0]:
path_database='./drive_PH/Colab Notebooks/' 
path_stop_word=path_database+'input/stopwords/'

pragraph_index='pragraph_index_20.csv'
Sentences='Sentences_20.csv'
Sentences_not_stops='Sentences_not_stops_20.csv'
lesk='lesk_20.csv'

#Steps

##Load 20newsgroups dataset

In [0]:
#@title Load 20newsgroups dataset


dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
documents[0]
dataset.filenames

In [0]:
#save_file_to_database(documents,path_database,pragraph_index,header_list)

In [0]:
header_list=['index','text']
#save_file_to_database(documents,'/',pragraph_index,header_list)

##nytimes_news_articles.

In [0]:
import pandas as pd
def courps_to_CSV_docs():
    #Reading the news articles file
    nyTimesFile = open('./nytimes_news_articles.txt', encoding='latin-1')
    nyTimesFile.seek(0)
    nyTimesV1 = nyTimesFile.readlines()
    nyTimesTemp = []
    nyTimesURL = []

    for i in range(0, len(nyTimesV1)-1):
        if re.findall('URL', nyTimesV1[i]) == []:
            sent = sent + nyTimesV1[i]
            if (re.findall('URL', nyTimesV1[i+1]) != []) and (i+1 < len(nyTimesV1)):
                nyTimesTemp.append(sent.strip())
        else:
            sent = ''
            nyTimesURL.append(nyTimesV1[i])

    for i in range(0, len(nyTimesTemp)):
        nyTimesTemp[i] = nyTimesTemp[i]+'articleID'+str(i)
    print(len(nyTimesTemp))
    header_list=['index','text']
    save_file_to_database(nyTimesTemp,path_database,pragraph_index,header_list)
    '''for i in range(1):
        print(i,"============================================")
        print("============================================")'''
    #nytimes = preProcessor(nyTimesTemp)
    print("============================================")
    #print(nytimes)

In [0]:
#courps_to_CSV_docs()

##Load paragraphs csv file

In [0]:
paragraphs=read_cvs_by_pands(path_database,pragraph_index,None,0)
paragraphs

##Convert paragraph to sentences

In [0]:

def paragraphs_to_sentece(pragraphs):
    sentenses_list=[]
    
    for index_p in  range(len(pragraphs)):
        #print(index_p)
        #print("pppppppppppppppppppp")
        #print(pragraphs[index_p])
        if pragraphs[index_p] is not None:
          #print("setnences")
          #p1=txt_pragraphs(pragraphs[index_p])
          setnences=pragraph_to_setnences(str(pragraphs[index_p]))
          #print("sssssssssssssssssssssssssss")
          #print("setnences")#,setnences)

          for indexs in range(len(setnences)):
              row=[]
              #print(setnences)
              row.append(index_p)
              row.append(indexs)
              row.append(setnences[indexs])
              sentenses_list.append(row)
    header_list=['index_P','index_sent','sentence']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#paragraphs_to_sentece(paragraphs.text)

##Load sentences CSV file

In [0]:
setences=read_cvs_by_pands(path_database,Sentences,None,0)
setences

##Conver sentence to word list
remove stop word 

In [0]:
def stopwords_list():
    stopwordsFile = open(path_stop_word+'stopwords.txt')
    stopwordsFile.seek(0)
    stopwordsV1 = stopwordsFile.readlines()
    stopwordsV2 = []
    for sent in stopwordsV1:
        sent.replace('\n', '')
        new_word = sent[0:len(sent) - 1]
        stopwordsV2.append(new_word.lower())
    return stopwordsV2

In [0]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
stopwordsV2=stopwords_list()
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(str):
   
            
    list_word=[]
    tokenizer = RegexpTokenizer("[\w']+")
    
    words=tokenizer.tokenize(str)
    for word in words:
        new_word = word.encode('ascii', 'ignore').decode('utf-8')
        if new_word != '':
    
            english_stops = set(stopwords.words('english'))
           
            list_word=[new_word for new_word in words if new_word.lower() not in english_stops
                       and new_word.lower() not in new_stop_words 
                       and new_word.lower() not in new_stop_words2 
                       and  not new_word.lower().isdigit() 
                       and new_word.lower() not in digits 
                       and new_word.lower() not in  numbers and word.lower() not in stopwordsV2
                       and new_word.lower() not in string.punctuation]
    
  
    
    return list_word#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

In [0]:

def  sentece_Not_stop_word(setences):
    #words_list=[]
    sentenses_list=[]
    
    for index_s in  range(len(setences)):
            
          #print("Sentence No. ",index_s,": ",setences.loc[index_s]['sentence'],"\n")
          words=remove_stopword_sentences(str(setences.loc[index_s]['sentence']))
          wordsent=''
          row=[]
          

          row.append(setences.loc[index_s]['index_P'])
          row.append(setences.loc[index_s]['index_sent'])
          row.append(words)
          sentenses_list.append(row)
    header_list=['index_P','index_sent','words_not_stop']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences_not_stops, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#sentece_Not_stop_word(setences)

##Load Sentences not stop word

In [0]:
df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
df_Sentences_not_stops

In [0]:
'''
this function for compute lesk of word in sentence
'''
def lesk_word_sentence(sentence,words):
    from nltk.wsd import lesk
    lesk_synset=''
    
    lesks_row= []
    lesks_name=[]
    #print(type(words))
    for i in range(len(words)):
        
        disambiguated=lesk(context_sentence=str(sentence), ambiguous_word=words[i].strip())
        
        
        if disambiguated is not None:
            lesks_row.append(disambiguated)
            lesks_name.append(disambiguated.name())
    
    
    return lesks_row,lesks_name
#disambiguated#lesk_synset

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#ll=['Well','"im"','sure','story','nad','seem','biased']
#lesk_word_sentence("Well i'm not sure about the story nad it did seem biased.", ll)
#lesk('I love dog', 'dog')



In [0]:


def sent_lesks():
    df_Sentences=read_cvs_by_pands(path_database,Sentences,None,0)
    #print(df_Sentences.columns)
    df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
    lesks=[]
    for i in range(len(df_Sentences_not_stops)):
        sent=[]
        full_words=df_Sentences_not_stops['words_not_stop'][i].replace("[",'').replace("]",'').replace("'",'').replace('"','')
        full_words_list=full_words.replace(",",'-')      
        words=full_words.split(',')
        sentense=df_Sentences['sentence'][i]
        
        sent.append(df_Sentences['index_P'][i])
        sent.append(df_Sentences['index_sent'][i])
        ss=lesk_word_sentence(sentense, words)
        
        sent.append(full_words_list)
        sent.append(ss[0])
        sent.append(ss[1])
        
        lesks.append(sent)
        
    header_list=['index_P','index_sent','full_word','lesks','lesks_name']
    df = pd.DataFrame(lesks, columns=header_list)
    
    df.to_csv(path_database+'lesk_20.csv', encoding='utf-8', index=False)  
    return df
    #return lesks
#sent_lesks()
    

##TF-IDF

###Creat lesk list per paragraph

In [0]:
def paragarph_to_lesk():
    pragraph_list=[]
    for p in range(2):#len(index_p_p)) :
        #print(p)
        l_names=l[l['index_P']==str(p)]['lesks_name']
        #print(len(l_names))

        l_name_one_paragraph=''
        paragraph_row=[]
        for i in range(len(l_names)):
            l_name_one_paragraph+=l_names.get_values()[i].replace('[','').replace(']',',').strip()

        if isNotBlank(l_name_one_paragraph.replace(',','')):


            paragraph_row.append(p)
            paragraph_row.append(l_name_one_paragraph)
            pragraph_list.append(paragraph_row)
        else:
            print(p,"pppppppppppppp")
    #print(pragraph_list)
    header_list=['index_P','lesk_list']
    df = pd.DataFrame(pragraph_list,columns=header_list)
    df.to_csv(path_database+'paragraph_lesk_20.csv', encoding='utf-8', index=False)  
    return df
    #df

###Load paragraph lesk

In [0]:
p_lesk=read_cvs_by_pands(path_database,'paragraph_lesk_20.csv',None,0)


###Calculate TF-IDF

In [0]:
def tfidf():
    cv_tfidf_lesk = TfidfVectorizer(analyzer='word',token_pattern="'"+'(?u)\\b\\w\\w+\\b\\.\\w\\.\\d\\d'+"'") #lesk
    #print(type(texts_lesk))
    cv_tfidf_fit_lesk=cv_tfidf_lesk.fit_transform(p_lesk['lesk_list']).toarray()
    df_tfidf_lesk=pd.DataFrame(cv_tfidf_fit_lesk,columns=cv_tfidf_lesk.get_feature_names(),index=p_lesk['index_P'])
    df_tfidf_lesk.to_csv(path_database+'tf_idf_lesk_table_20.csv')
    #save_file_to_database(texts_lesk,path_database,lesk_paragraph,lesk_paragraph_list)
    return df_tfidf_lesk

###Load TF-IDF

In [0]:
tfidf_list=read_cvs_by_pands(path_database,'tf_idf_lesk_table_20.csv',None,0).index

#Word2Vec

In [0]:
import gensim
import gensim.models
from gensim.models import Word2Vec
from gensim.models import word2vec
import logging
from gensim.models import Phrases

In [0]:
#https://taylorwhitten.github.io/blog/word2vec

import gensim
import gensim.models
from gensim.models import Word2Vec
from gensim.models import word2vec
import logging
from gensim.models import Phrases

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 50   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


model = word2vec.Word2Vec(p_lesk['lesk_list'], workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)
model_name = "lesk2vec.bin"
model.wv.save_word2vec_format(path_database+model_name, binary=True)
#new_model = gensim.models.Word2Vec.load('lesk2vec')


In [0]:
ll=[['ff','rtert','fgdfgf'],['ff','trt','hgj'],['trt','rtert','tyt']]

model2 = gensim.models.KeyedVectors.load_word2vec_format(ll, binary=False)

In [0]:
#model.most_similar('yea.r.01',  topn=15)
new_model =  gensim.models.KeyedVectors.load_word2vec_format(path_database+'lesk2vec.bin', binary=True)

In [0]:
new_model.wv.vocab

In [0]:
l=read_cvs_by_pands(path_database,'lesk_20.csv',None,0)
l

In [0]:
p=read_cvs_by_pands(path_database,'pragraph_index_20.csv',None,0)
#index_p_p=p['index']


In [0]:
l_names=[]
for i in range(len(index_p_p)) :
    l_names=l[l['index_P']==str(i)]['lesks_name']
print(l_names)

In [0]:
len(index_p_p)
l_names=l[l['index_P']==str(11313)]['lesks_name']
l_names

In [0]:
print(p_lesk['lesk_list'][1:10])

In [0]:
print (len(all_feature))

#Semantic similarty Lesk

In [15]:
def isNotBlank (myString):
    if myString and myString.strip():
        #myString is not None AND myString is not empty or blank
        return True
    #myString is None OR myString is empty or blank
    return False
isNotBlank(' ')

False

In [0]:

def similarity_by_infocontent(sense1, sense2, option):
    #sense1="Synset('"+sense1+"')"
    #sense2="Synset('"+sense2+"')"
    sense1 = wn.synset(sense1)
    sense2 = wn.synset(sense2)
    #print(sense1,sense2)
    """ Returns similarity scores by information content. """
    #if sense1.pos != sense2.pos: # infocontent sim can't do diff POS.
        #return 0

    info_contents = ['ic-bnc-add1.dat', 'ic-bnc-resnik-add1.dat', 
                     'ic-bnc-resnik.dat', 'ic-bnc.dat', 

                     'ic-brown-add1.dat', 'ic-brown-resnik-add1.dat', 
                     'ic-brown-resnik.dat', 'ic-brown.dat', 

                     'ic-semcor-add1.dat', 'ic-semcor.dat',

                     'ic-semcorraw-add1.dat', 'ic-semcorraw-resnik-add1.dat', 
                     'ic-semcorraw-resnik.dat', 'ic-semcorraw.dat', 

                     'ic-shaks-add1.dat', 'ic-shaks-resnik.dat', 
                     'ic-shaks-resnink-add1.dat', 'ic-shaks.dat', 

                     'ic-treebank-add1.dat', 'ic-treebank-resnik-add1.dat', 
                     'ic-treebank-resnik.dat', 'ic-treebank.dat']

    if option in ['res', 'resnik']:
        #return wn.res_similarity(sense1, sense2, wnic.ic('ic-bnc-resnik-add1.dat'))
        #print('simRe snik (c1,c2) = -log p(lso(c1,c2)) = IC(lso(c1,c2)')
        return wn.res_similarity(sense1, sense2, wnic.ic('ic-treebank-resnik-add1.dat'))
    #return min(wn.res_similarity(sense1, sense2, wnic.ic(ic)) \
    #             for ic in info_contents)

    elif option in ['jcn', "jiang-conrath"]:
        #return wn.jcn_similarity(sense1, sense2, wnic.ic('ic-bnc-add1.dat'))
        #print('sim(jcn) (c1,c2 )= (IC(c1) + IC(c2 )) - 2IC(lso(c1,c2 ))')
        return wn.jcn_similarity(sense1, sense2, wnic.ic('ic-treebank.dat'))

    elif option in ['lin']:
        #return wn.lin_similarity(sense1, sense2, wnic.ic('ic-bnc-add1.dat'))
        #print('sim(lin) (c1,c2)=(2IC(lso(c1,c2 )))/(IC(c1)+IC(c2))')
        return wn.lin_similarity(sense1, sense2, wnic.ic('ic-treebank.dat'))

def sim(sense1, sense2, option="path"):
    """ Calculates similarity based on user's choice. """
    option = option.lower()
    if option.lower() in ["path", "path_similarity", 
                        "wup", "wupa", "wu-palmer", "wu-palmer",
                        'lch', "leacock-chordorow"]:
        return similarity_by_path(sense1, sense2, option) 
    elif option.lower() in ["res", "resnik",
                          "jcn","jiang-conrath",
                          "lin"]:
        return similarity_by_infocontent(sense1, sense2, option)

def max_similarity(context_sentence, ambiguous_word, option="path", 
                   pos=None, best=True):
    """
    Perform WSD by maximizing the sum of maximum similarity between possible 
    synsets of all words in the context sentence and the possible synsets of the 
    ambiguous words (see http://goo.gl/XMq2BI):
    {argmax}_{synset(a)}(\sum_{i}^{n}{{max}_{synset(i)}(sim(i,a))}
    """
    result = {}
    for i in wn.synsets(ambiguous_word):
        try:
            if pos and pos != str(i.pos()):
                continue
        except:
            if pos and pos != str(i.pos):
                continue
        result[i] = sum(max([sim(i,k,option) for k in wn.synsets(j)]+[0]) \
                        for j in word_tokenize(context_sentence))

    if option in ["res","resnik"]: # lower score = more similar
        result = sorted([(v,k) for k,v in result.items()])
    else: # higher score = more similar
        result = sorted([(v,k) for k,v in result.items()],reverse=True)
    #print (result)
    if best: return result[0][1];
    return result

In [19]:
'''
calculate simantic simelart for Dimensionality reduction vector
say vector is n element [n1,n2,n3,.....nm], data frame row=n,col=n
sim(n[row],n[col])if if row != col:
option is sim method like res,lin,jcn ...... for IC

'''
#print(type(tfidf_feature_names))
#tfidf_feature_names
import nltk
nltk.download('wordnet')
nltk.download('wordnet_ic')
def sim_docs_lesk(df_freq,option):
    
    
    series=list(df_freq)#pd.Series(data=lesk_vec)
    #series.drop_duplicates()
    synset_lesk_noDuplicates= series#.tolist()    
    #df_all_synset_lesk = pd.DataFrame(index=series, columns=series )
    #df_all_synset_lesk = pd.DataFrame(columns=series ) **********
    #print ("synset_lesk_noDuplicates",len(synset_lesk_noDuplicates))
    
    for row in range(len(synset_lesk_noDuplicates)):
        #try:
            row_csv=[]
            row_csv.append(synset_lesk_noDuplicates[row])
            print("1",row_csv)
            #data_row=[]**********
            for col in range(len(synset_lesk_noDuplicates)):

                #if row < col:
                try:
                    sim=similarity_by_infocontent(series[row],series[col],option)
                    if sim is not None:
                        #data_row.append(sim)***********
                        row_csv.append(sim)
                    else:
                        #data_row.append(0)*************
                        #print(len(data_row))
                        row_csv.append(0)
                except  Exception as inst:
                    #data_row.append(0)*********
                    row_csv.append(0)
                    #print(type(inst))    # the exception instance
                    #print(inst.args)     # arguments stored in .args
                    #print(inst)          # __str__ allows args to be printed directly,
                    pass
                    #print("Ex")

            #print("2",row_csv)
            add_row(row_csv,path_database,"res_sem_sim_table.csv")
            #df_all_synset_lesk.loc[series[row]]=data_row**********
       
    #return df_all_synset_lesk



[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /content/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [0]:
f=open(path_database+'tf_idf_lesk_table_20.csv')
all_feature=f.readline().replace("'",'').split(',')[1:]
headers=all_feature.copy()

headers.insert( 0, " ")
#add_row(headers,path_database,"res_sem_sim_table.csv")
#print (all_feature)
#print (headers)
#print (len(all_feature))
#df_sim_sem_lesk=
sim_docs_lesk(all_feature,'res')
#df_sim_sem_lesk.to_csv(path_database+'res_sem_sim_table.csv')

1 ['1760s.n.01']


In [18]:
res_sem_sim_table=read_cvs_by_pands(path_database,'res_sem_sim_table.csv',None,0)
res_sem_sim_table

,,1760s.n.01,1830s.n.01,1840s.n.01,1850s.n.01,1870s.n.01,1900s.n.01,aachen.n.01,aardvark.n.01,aaron.n.02,...,zoological.a.02,zoologist.n.01,zoom.v.02,zoroaster.n.01,zoroastrian.n.01,zoroastrianism.n.01,zu.n.01,zurich.n.01,zurvanism.n.02,zymosis.n.01
0,1760s.n.01,13.523224,7.652986,7.652986,7.652986,7.652986,7.652986,-0.0,-0.0,0.622355,...,0,-0.0,0,-0.0,-0.0,0.622355,0.622355,-0.0,0.622355,-0.0
